In [8]:
import ffmpeg
import os

def process_video(input_video_path, output_video_path_prefix):
    # Get the video duration using ffprobe
    probe = ffmpeg.probe(input_video_path, v='error', select_streams='v:0', show_entries='format=duration')
    duration = float(probe['format']['duration'])

    if duration >= 2.976044:
        # Trim the video to the first 2.976044 seconds
        first_video = ffmpeg.input(input_video_path, t=2.976044)

        # Create a silent audio stream for the first video
        silent_audio = ffmpeg.input('anullsrc=r=44100:cl=stereo', t=2.976044, f='lavfi')

        # Concatenate the first video and silent audio
        first_video = ffmpeg.concat(first_video, silent_audio, v=1, a=1)

        # Save the first 2.976044 seconds as the first video
        first_output_path = output_video_path_prefix + "_first.mp4"
        first_video.output(first_output_path).run(overwrite_output=True)

        # Calculate the remaining duration
        remaining_duration = duration - 2.976044

        if remaining_duration > 0:
            # Trim the remaining part of the video
            second_video = ffmpeg.input(input_video_path, ss=2.976044, t=remaining_duration)

            # Create a silent audio stream for the second video
            silent_audio = ffmpeg.input('anullsrc=r=44100:cl=stereo', t=remaining_duration, f='lavfi')

            # Concatenate the second video and silent audio
            second_video = ffmpeg.concat(second_video, silent_audio, v=1, a=1)

            # Save the remaining duration as the second video
            second_output_path = output_video_path_prefix + "_second.mp4"
            second_video.output(second_output_path).run(overwrite_output=True)
    else:
        # Calculate the duration to pad
        padding_duration = 2.976044 - duration

        # Create a padding video with a black screen
        padding = (
            ffmpeg.input('anullsrc=r=44100:cl=stereo', t=padding_duration, f='lavfi')  # Silent audio
            .output('color=c=black:s=1280x720:r=25:d=' + str(padding_duration), f='lavfi')  # Black screen video
            .compile()
        )

        # Concatenate the input video and padding video
        input_video = ffmpeg.input(input_video_path)
        final_video = ffmpeg.concat(input_video, padding, v=1, a=1)

        # Save the video with padding
        final_output_path = output_video_path_prefix + "_padded.mp4"
        final_video.output(final_output_path).run(overwrite_output=True)

# Example usage
input_video_path = 'origVid.mp4'
output_video_path = 'output.mp4'
process_video(input_video_path, output_video_path)





ffmpeg: /home/sigfred/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /home/sigfred/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg: /home/sigfred/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /lib/x86_64-linux-gnu/libcaca.so.0)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype 

In [9]:
import subprocess
import os
def get_video_duration(video_path):
    command = ['ffprobe', '-i', video_path, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=p=0']
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    if result.returncode == 0:
        duration = float(result.stdout.strip())
        return duration
    else:
        print('Error:', result.stderr)
        return None

video_path = os.path.join('output.mp4_first.mp4')  # Replace with your video file path
duration = get_video_duration(video_path)

if duration is not None:
    print(f'The duration of the video is: {duration} seconds')

The duration of the video is: 3.018 seconds
